In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from imblearn.over_sampling import SMOTE

2025-04-28 18:57:56.783648: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-28 18:57:59.252343: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-28 18:57:59.829547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745855880.777654   47170 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745855880.938511   47170 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745855882.950894   47170 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# Loading the cleaned data from the CSV file for training
cancer_data = pd.read_csv('../datasets/cleaned_cervical_cancer_data.csv')

In [3]:
# We first convert our data frame into numpy arrays
input_array = np.array(cancer_data.drop(columns=["Biopsy"])).astype("float32")
target_array = np.array(cancer_data["Biopsy"]).astype("float32")

In [4]:
# Apply SMOTE (Synthetic Minority Over-sampling Technique) to balance the dataset
# SMOTE increases the samples of the minority class to achieve a balanced distribution

smote = SMOTE(random_state=12)  
# Generate a balanced dataset with SMOTE
X, y = smote.fit_resample(input_array, target_array)  

In [5]:
#We split the data into training and test sets
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=12)


In [6]:
#Since some algorithms require scaled data for better performance, we are going to also prepare scaled data
# scaler = MinMaxScaler();
# scaler.fit(X_train);
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# Build MLP Model with Optimizations
model = tf.keras.Sequential([
    # Input layer with proper Input layer usage
    layers.InputLayer(input_shape=(X_train.shape[1],)),  # or layers.Input(shape=(X_train.shape[1],))
    
     # Hidden Layer 1 with Dropout and BatchNormalization
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),  # Apply dropout to prevent overfitting
    
    # Hidden Layer 1 with Dropout and BatchNormalization
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),  # Apply dropout to prevent overfitting
    
    # Hidden Layer 2 with Dropout and BatchNormalization
    layers.Dense(32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),  # Apply dropout to prevent overfitting
    
    # Output layer for binary classification
    layers.Dense(1, activation='sigmoid')  # 1 neuron for binary classification
])


/home/emperor/Coding/Python/ML/fyp/myvenv/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
2025-04-28 18:58:33.197901: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
# Compile the model with AdamW optimizer
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001),  # AdamW optimizer for better performance
    loss='binary_crossentropy',  # Binary classification
    metrics=['accuracy']
)

In [9]:
# Add EarlyStopping and ReduceLROnPlateau callbacks for better training control
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

In [10]:
# Summary of the model architecture
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         3,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,465 (56.50 KB)

 Trainable params: 14,017 (54.75 KB)

 Non-trainable params: 448 (1.75 KB)

In [11]:
# Train the model (example)
history = model.fit(
    X_train, y_train, 
    epochs=50, 
    batch_size=32, 
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.5615 - loss: 0.8443 - val_accuracy: 0.6589 - val_loss: 0.6510 - learning_rate: 0.0010
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6285 - loss: 0.7018 - val_accuracy: 0.6563 - val_loss: 0.6150 - learning_rate: 0.0010
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6902 - loss: 0.6126 - val_accuracy: 0.7313 - val_loss: 0.5780 - learning_rate: 0.0010
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7315 - loss: 0.5239 - val_accuracy: 0.8140 - val_loss: 0.5378 - learning_rate: 0.0010
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7513 - loss: 0.5146 - val_accuracy: 0.8036 - val_loss: 0.4983 - learning_rate: 0.0010
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7456 - loss: 0.5013 - val_accuracy: 0.8398 - val_loss: 0.4624 - learning_rate: 0.0010
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7689 - loss: 0.4820 - val_accuracy:

In [12]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9190 - loss: 0.2107 
Test accuracy: 0.9147287011146545


In [13]:
# 8. Save model
model.save('./saved_models/cervical_cancer_model.h5')

In [14]:
# Load
model = tf.keras.models.load_model('./saved_models/cervical_cancer_model.h5')

# Convert
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save
with open('./saved_models/cervical_cancer_model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpd7dq1su8/assets


INFO:tensorflow:Assets written to: /tmp/tmpd7dq1su8/assets


Saved artifact at '/tmp/tmpd7dq1su8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 24), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140018931342928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140018931340432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140018820761744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140018820762512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140018931343120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140018931342544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140018820764816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140018820763856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140018432224592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140018432225360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140018820762320: TensorS

W0000 00:00:1745855933.381001   47170 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1745855933.381050   47170 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-04-28 18:58:53.381545: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpd7dq1su8
2025-04-28 18:58:53.383092: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-04-28 18:58:53.383108: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpd7dq1su8
I0000 00:00:1745855933.396119   47170 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-04-28 18:58:53.398852: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-04-28 18:58:53.478731: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpd7dq1su8
2025-04-28 18:58:53.501634: I tensorflow/cc/saved_model/loader.cc:471] SavedModel 